In [89]:
from __future__ import division
import pandas as pd
import numpy as np

import graphlab as gl

from sklearn.linear_model import LogisticRegression as LR

import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')

In [2]:
import src.metadata_handler as mdh
import src.clean_db as cdb

%autoreload 2

In [53]:
# raw_df = pd.read_csv('data/metadata.csv')

In [54]:
# raw_df = raw_df.rename(columns = {'Unnamed: 0':'photo_id'})

In [55]:
# raw_df.head().T

In [57]:
# df = raw_df[['keywords', 'photo_id','date_created','time_created','copyright_notice','file_path']]

In [58]:
# df.copyright_notice.unique()

## WITH LOCAL PHOTOS:

### create my dataframe

In [76]:
mdh.build_json_database('first_sample','data/testing/first_sampl.json')
cdb.main('data/testing/first_sampl.json', 'data/testing/first_sampl.csv')

raw_df = pd.read_csv('data/testing/first_sampl.csv')
# raw_df.head(3).T

('WARNING: problems with charset recognition', "'\\x1b'")
0
('WARNING: problems with charset recognition', "'\\x1b'")
0
('WARNING: problems with charset recognition', "'\\x1b'")
0
('WARNING: problems with charset recognition', "'\\x1b'")
0
('WARNING: problems with charset recognition', "'\\x1b'")
0
('WARNING: problems with charset recognition', "'\\x1b'")
0
('WARNING: problems with charset recognition', "'\\x1b'")
0
('WARNING: problems with charset recognition', "'\\x1b'")
0
('WARNING: problems with charset recognition', "'\\x1b'")
0
('WARNING: problems with charset recognition', "'\\x1b'")
10
('WARNING: problems with charset recognition', "'\\x1b'")
10
('WARNING: problems with charset recognition', "'\\x1b'")
10
('WARNING: problems with charset recognition', "'\\x1b'")
10
('WARNING: problems with charset recognition', "'\\x1b'")
10
('WARNING: problems with charset recognition', "'\\x1b'")
10
('WARNING: problems with charset recognition', "'\\x1b'")
10
('WARNING: problems with charset 

In [82]:
df = raw_df.rename(columns = {'Unnamed: 0':'photo_id'})
# df['image'] = image_pdSeries <-- this turns images into dicts (can't change them back)

# add dummy cols to dataframe:
df = pd.concat([df, pd.get_dummies(df.keywords)], axis=1)

df.loc[pd.isnull(df.copyright_notice), 'copyright_notice'] = "not_provided"

# Drop unneeded columns
df = df.drop(['by_line', 'caption_abstract',
              'contact','object_name','sub_location',
              'supplemental_category','keywords', "[u'camera check']", "[u'marmot']", "[u'deer']"],
             axis = 1)

# # take only rows where copyright is properly entered
# df = df.loc[df['copyright_notice'] == 'Conservation Northwest']

df.head(3).T

,0,1,2
photo_id,0,1,2
copyright_notice,Conservation Northwest,not_provided,Conservation Northwest
date_created,2015-05-30,None,2015-06-04
file_path,first_sample/EK000004-2.JPG,first_sample/EK000004.JPG,first_sample/EK000010-2.JPG
time_created,11:49:52,None,14:54:40
[],0,0,0
[u'Camera Check'],1,0,0
[u'hoary marmot'],0,0,1
[u'mule deer'],0,0,0
[u'unidentified'],0,0,0


### play with graphlab

graphlab example of feature extraction:
```python
>>> data = graphlab.SFrame('https://static.turi.com/datasets/mnist/sframe/train6k')
>>> net = graphlab.deeplearning.get_builtin_neuralnet('mnist')
>>> m = graphlab.neuralnet_classifier.create(data,
...                                          target='label',
...                                          network=net,
...                                          max_iterations=3)
>>> # Now, let's extract features from the last layer
>>> data['features'] = m.extract_features(data)
>>> # Now, let's build a new classifier on top of extracted features
>>> m = graphlab.classifier.create(data,
...                                          features = ['features'],
...                                          target='label')
```
Now, let’s see how to load the ImageNet model, and use it for extracting features after resizing the data:
```python

>>> imagenet_model = graphlab.load_model('https://static.turi.com/products/graphlab-create/resources/models/python2.7/imagenet_model_iter45')
>>> data['image'] = graphlab.image_analysis.resize(data['image'], 256, 256, 3, decode=True)
>>> data['imagenet_features'] = imagenet_model.extract_features(data)

```


In [114]:
data2 = gl.SFrame('https://static.turi.com/datasets/mnist/sframe/train6k')

image_col = data2['image']

data2.dtype

<bound method SFrame.dtype of Columns:
	label	int
	image	Image

Rows: 6000

Data:
+-------+----------------------+
| label |        image         |
+-------+----------------------+
|   5   | Height: 28 Width: 28 |
|   8   | Height: 28 Width: 28 |
|   1   | Height: 28 Width: 28 |
|   4   | Height: 28 Width: 28 |
|   2   | Height: 28 Width: 28 |
|   7   | Height: 28 Width: 28 |
|   0   | Height: 28 Width: 28 |
|   2   | Height: 28 Width: 28 |
|   5   | Height: 28 Width: 28 |
|   9   | Height: 28 Width: 28 |
+-------+----------------------+
[6000 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.>

### turn dataframe into SFrame

In [83]:
data = gl.SFrame(df)

# GET IMAGES:
img_sframe = gl.image_analysis.load_images('first_sample', "auto", with_path=False,
                                                    recursive=True)

# ADD IMAGES TO data
data.add_column(img_sframe['image'], name='image')

# Filter for desired rows ('copyright_notice' == 'Conservation Northwest')
data = data[data['copyright_notice'] == 'Conservation Northwest']

In [85]:
# Get Imagenet pretrained CNN
imagenet_model = gl.load_model('https://static.turi.com/products/graphlab-create\
                                        /resources/models/python2.7/imagenet_model_iter45')
# Convert photos to proper size for imagenet
data['image'] = gl.image_analysis.resize(data['image'], 256, 256, 3, decode=True)

# Get features from CNN
data['imagenet_features'] = imagenet_model.extract_features(data)

Downloading https://static.turi.com/products/graphlab-create                                        /resources/models/python2.7/imagenet_model_iter45/dir_archive.ini to /var/tmp/graphlab-ophidian/33361/629e0223-1cf0-4979-ac68-d4dc1de40ab6.ini

Downloading https://static.turi.com/products/graphlab-create                                        /resources/models/python2.7/imagenet_model_iter45/objects.bin to /var/tmp/graphlab-ophidian/33361/1624c76d-3838-4ec7-b2c8-a86e315c40e6.bin

# Create Model!

In [113]:
log_regr = LR(penalty='l2', dual=False, tol=0.0001, C=1.0,
           fit_intercept=True, intercept_scaling=1,
           class_weight=None, random_state=None, solver='liblinear',
           max_iter=100, multi_class='ovr', verbose=0,
           warm_start=False, n_jobs=2)

X = data['imagenet_features']
y = data["[u'mule deer']"]

# fit model to data
model = log_regr.fit(X,y)
pred = model.predict(X)
prob = model.predict_proba(X)

print'true | pred | prob [mule deer = true]'
for a,b,c in zip(y, pred, prob):
    print a,'  ',b,'  ',c[1]

print model.score(X,y)

true | pred | prob [mule deer = true]
0.0    0.0    0.00226344941121
0.0    0.0    0.0433000493441
0.0    0.0    0.00980105000864
0.0    0.0    0.477570801839
0.0    0.0    0.498634624133
0.0    0.0    0.0618667062038
1.0    1.0    0.946675543696
1.0    1.0    0.987679561889
1.0    0.0    0.498634624133
1.0    1.0    0.990925428388
0.0    0.0    0.00319358644227
0.0    0.0    0.00319358644227
1.0    1.0    0.501632769344
1.0    0.0    0.477570801839
1.0    1.0    0.990925428388
0.0    1.0    0.501632769344
0.0    0.0    0.00319358644227


ValueError: Expected array-like (array or non-string sequence), got dtype: float
Rows: 17
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0]